In [1]:
import sys
sys.path.append('../../..')

In [ ]:
import numpy as np
from hyperbolicity.tree_fitting_methods.neighbor_joining import NJ
from hyperbolicity.tree_fitting_methods.treerep import TreeRep
from hyperbolicity.tree_fitting_methods.hccfit import HccLinkage
from hyperbolicity.tree_fitting_methods.gromov import gromov_tree, buneman_extraction
from hyperbolicity.delta import compute_hyperbolicity
from hyperbolicity.tree_fitting_methods.layering_tree import layering_approx_tree
from hyperbolicity.tree_fitting_methods.deltazero import deltazero_tree
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout
import torch


plt.rcParams['text.usetex'] = True


import seaborn as sns
sns.set_theme(style="white")

In [3]:
from matplotlib.colors import Normalize
from matplotlib.colors import TwoSlopeNorm

def draw_graph(n, distances, tree, ax, title='Tree', fixed_pos=None, fixed_nodes=None,
               node_size=500, edge_color='gray',root=0,show_root=True,
               orig_color = '#FE6100',
               int_color  = '#648FFF',
               root_color = '#DC267F',
               cmap=sns.color_palette("Blues", as_cmap=True),
               with_labels=True,
               scale_width=6,
               fs=24,
               vmin=0, vmax=2):

    orig_nodes_nj = [v for v in tree.nodes() if v < n]
    int_nodes_nj  = [v for v in tree.nodes() if v >= n]

    # Set edge weights from the distance matrix M
    edge_values = {}
    for (u, v) in tree.edges():
        tree[u][v]['weight_layout'] = 1./(.0001+distances[u, v])
        edge_values[(u,v)]= distances[u, v]

    norm = TwoSlopeNorm(vmin=vmin, vcenter=1, vmax=vmax)
    edge_colors = [cmap(norm(edge_values[(u,v)])) for (u,v) in tree.edges()]
    # Get positions for the nodes
    pos = nx.spring_layout(tree, pos=fixed_pos, fixed=fixed_nodes, weight='weight_layout',  iterations=1000)
    #pos = nx.spring_layout(tree, pos=fixed_pos, fixed=fixed_nodes, iterations=1000)

    nx.draw_networkx_edges(tree, pos, ax=ax, edge_color=edge_colors, width=scale_width)
    #nx.draw_networkx_edges(tree, pos, ax=ax)
    nx.draw_networkx_nodes(tree, pos,
                        nodelist=int_nodes_nj,
                        node_color=int_color,  node_size=node_size/2, edgecolors='k',linewidths=1,
                        ax=ax)
    nx.draw_networkx_nodes(tree, pos,
                        nodelist=orig_nodes_nj,
                        node_color=orig_color, node_size=node_size, edgecolors='k',linewidths=2,
                        ax=ax)

    if show_root:
        nx.draw_networkx_nodes(tree, pos,
                        nodelist=[root] if root in tree else [],
                        node_color=root_color,
                        node_size=node_size, ax=ax)

    if with_labels:
        nx.draw_networkx_labels(tree, pos,
                                labels={v: str(v) for v in tree.nodes()},
                                font_size=20, font_color='black',
                                ax=ax)

    ax.set_title(title, pad=10,fontsize=fs)
    ax.axis('off')


def plot_distances(distances, ax, vmin=0, vmax=10,
                   cmap=sns.diverging_palette(230, 20, as_cmap=True)):
    # Generate a mask for the upper triangle
    mask = np.triu(np.ones_like(distances, dtype=bool),k=1)

    # Draw the heatmap with the mask and correct aspect ratio
    #sns.heatmap(distances, mask=mask, cmap=cmap, vmin=vmin, vmax=vmax, center=0,
    #            square=True, linewidths=.5, cbar_kws={"shrink": .6},ax=ax)
    divnorm = TwoSlopeNorm(vmin=vmin, vcenter=1, vmax=vmax)
    sns.heatmap(distances, mask=mask, cmap=cmap, norm=divnorm,
                square=True, linewidths=.7, cbar=False,ax=ax)

In [4]:
def create_sbm(n1=50, n2=50, p_in=0.3, p_out=0.05, seed=None):
    if seed is not None:
        np.random.seed(seed)
    sizes = [n1, n2]  # Community sizes
    prob_matrix = [[p_in, p_out], [p_out, p_in]]  # Block probability matrix
    G = nx.stochastic_block_model(sizes, prob_matrix, seed=seed)

    return G

from itertools import combinations, groupby
import random

def gnp_random_connected_graph(n, p):
    """
    Generates a random undirected graph, similarly to an Erdős-Rényi
    graph, but enforcing that the resulting graph is conneted
    """
    edges = combinations(range(n), 2)
    G = nx.Graph()
    G.add_nodes_from(range(n))
    if p <= 0:
        return G
    if p >= 1:
        return nx.complete_graph(n, create_using=G)
    for _, node_edges in groupby(edges, key=lambda x: x[0]):
        node_edges = list(node_edges)
        random_edge = random.choice(node_edges)
        G.add_edge(*random_edge)
        for e in node_edges:
            if random.random() < p:
                G.add_edge(*e)
    return G

# generate graph and compute stuff

In [ ]:
root =2

N = 10
random_graph = nx.random_geometric_graph(N, 0.5,seed=11)
#N = 4*4
#random_graph = nx.grid_graph(dim=(4,4))
#random_graph = nx.convert_node_labels_to_integers(random_graph, first_label = 0)
#N1=10
#N2=10
#N = N1 + N2
#random_graph= create_sbm(n1=N1, n2=N2, p_in=0.5, p_out=0.06, seed=42)
#N=10
#random_graph = gnp_random_connected_graph(N,.1)

distances = nx.floyd_warshall_numpy(random_graph)

tree_nj = NJ(distances)
tree_nj_distances = nx.floyd_warshall_numpy(tree_nj)

tree_TR = TreeRep(distances)
tree_TR.learn_tree()
tree_TR_distances = nx.floyd_warshall_numpy(tree_TR.G)

tree_hcc = HccLinkage(distances)
tree_hcc.fit_tree(root)
tree_hcc_distance = nx.floyd_warshall_numpy(tree_hcc.G)

gromov_distance = gromov_tree(distances, root)
tree_gromov_full = buneman_extraction(gromov_distance, root)
distance_gromov_tree = nx.floyd_warshall_numpy(tree_gromov_full)

layering_tree = layering_approx_tree(random_graph, root)
distance_layering = nx.floyd_warshall_numpy(layering_tree)

opt_tree = deltazero_tree(torch.tensor(distances), root, 1e-2, 1, N, 1000, .01, 2000, False, True)
opt_tree_full = buneman_extraction(opt_tree, root)
distance_deltazero = nx.floyd_warshall_numpy(opt_tree_full)


Early stopping triggered:  10%|▉         | 193/2000 [00:01<00:12, 143.36it/s]                        


display

In [ ]:
node_size = 400
size_fig = 5
cmap_dist = sns.diverging_palette(180,25,s=100,l=60, center="dark", as_cmap=True)
#sns.color_palette(palette='bone', as_cmap=True)
#sns.diverging_palette(220,30, l=65, center="dark", as_cmap=True)
# #sns.color_palette("icefire", as_cmap=True)
vmin = 0
vmax = 4
fs=44

# let's plot the trees
fig, axes = plt.subplots(2, 7, figsize=(7 * size_fig, 2 * size_fig))

# colorblind‑friendly
orig_color = '#648FFF'  # blue
int_color = '#FE6100'  # orange
root_color = '#DC267F'  # colorblind-friendly green

# 1) Original points with labels
ax = axes[0, 0]
fixed_positions = nx.spring_layout(random_graph, seed=0)
fixed_nodes = fixed_positions.keys()

draw_graph(N, distances, random_graph, ax, title='Original Graph', fixed_pos=fixed_positions, fixed_nodes=fixed_nodes,
           cmap=cmap_dist, node_size=node_size+100, show_root=False, vmax=vmax, fs=fs)

ax = axes[1, 0]
plot_distances(distances, ax, vmin=vmin, vmax=vmax, cmap=cmap_dist)

# 2) Chepoi
ax = axes[0, 1]
draw_graph(N, distance_layering, layering_tree, ax, title='LayeringTree', fixed_pos=fixed_positions, fixed_nodes=fixed_nodes,
           cmap=cmap_dist, node_size=node_size, root=root, vmax=vmax, fs=fs, with_labels=False)
ax = axes[1, 1]
plot_distances(distance_layering[:N, :N], ax, vmin=vmin, vmax=vmax, cmap=cmap_dist)
distortion = np.abs(distances - distance_layering[:N, :N]).max()
ax.text(0.5, -0.2, f"Distortion: {distortion:.2f}", ha='center', va='center', transform=ax.transAxes, fontsize=fs)

# 3) TreeRep
ax = axes[0, 2]
draw_graph(N, tree_TR_distances, tree_TR.G, ax, title='TreeRep', fixed_pos=fixed_positions, fixed_nodes=fixed_nodes,
           cmap=cmap_dist, node_size=node_size, root=root, vmax=vmax, fs=fs, with_labels=False)
ax = axes[1, 2]
plot_distances(tree_TR_distances[:N, :N], ax, vmin=vmin, vmax=vmax, cmap=cmap_dist)
distortion = np.abs(distances - tree_TR_distances[:N, :N]).max()
ax.text(0.5, -0.2, f"Distortion: {distortion:.2f}", ha='center', va='center', transform=ax.transAxes, fontsize=fs)

# 4) Gromov
ax = axes[0, 3]
l = np.argsort(tree_gromov_full.nodes)
draw_graph(N, distance_gromov_tree[l], tree_gromov_full, ax, title='Gromov', fixed_pos=fixed_positions, fixed_nodes=fixed_nodes,
           cmap=cmap_dist, node_size=node_size, root=root, vmax=vmax,fs=fs, with_labels=False)
ax = axes[1, 3]
plot_distances(distance_gromov_tree[l][:N, l[:N]], ax, vmin=vmin, vmax=vmax, cmap=cmap_dist)
distortion = np.abs(distances - distance_gromov_tree[l][:N, l[:N]]).max()
ax.text(0.5, -0.2, f"Distortion: {distortion:.2f}", ha='center', va='center', transform=ax.transAxes, fontsize=fs)

# HCC
ax = axes[0, 4]
l = np.argsort(tree_hcc.G.nodes)
draw_graph(N, tree_hcc_distance[l], tree_hcc.G, ax, title='HCC', fixed_pos=fixed_positions, fixed_nodes=fixed_nodes,
           node_size=node_size, cmap=cmap_dist, root=root, vmax=vmax,fs=fs, with_labels=False)
ax = axes[1, 4]
plot_distances(tree_hcc.d_T, ax, vmin=vmin, vmax=vmax, cmap=cmap_dist)
distortion = np.abs(distances - tree_hcc.d_T).max()
ax.text(0.5, -0.2, f"Distortion: {distortion:.2f}", ha='center', va='center', transform=ax.transAxes, fontsize=fs)

# 5) Neighbor‑Joining
ax = axes[0, 5]
draw_graph(N, tree_nj_distances, tree_nj, ax, title='Neighbor Joining', fixed_pos=fixed_positions, fixed_nodes=fixed_nodes,
           cmap=cmap_dist, node_size=node_size, root=root, vmax=vmax, fs=fs, with_labels=False)
ax = axes[1, 5]
plot_distances(tree_nj_distances[:N, :N], ax, vmin=vmin, vmax=vmax, cmap=cmap_dist)
distortion = np.abs(distances - tree_nj_distances[:N, :N]).max()
ax.text(0.5, -0.2, f"Distortion: {distortion:.2f}", ha='center', va='center', transform=ax.transAxes, fontsize=fs)




# 7) HDTree
ax = axes[0, 6]
l = np.argsort(opt_tree_full.nodes)
draw_graph(N, distance_deltazero[l], opt_tree_full, ax, title='StrongZero (ours)', fixed_pos=fixed_positions, fixed_nodes=fixed_nodes,
           cmap=cmap_dist, node_size=node_size, root=root, vmax=vmax, fs=fs, with_labels=False)

ax = axes[1, 6]
plot_distances(distance_deltazero[l][:N, l[:N]], ax, vmin=vmin, vmax=vmax, cmap=cmap_dist)
distortion = np.abs(distances - distance_deltazero[l][:N, l[:N]]).max()
ax.text(0.5, -0.2, f"Distortion: {distortion:.2f}", ha='center', va='center', transform=ax.transAxes, fontsize=fs)

cbar_ax = fig.add_axes([1.01, 0.15, 0.02, 0.7])  # Position: [gauche, bas, largeur, hauteur]
divnorm = TwoSlopeNorm(vmin=vmin, vcenter=1, vmax=vmax)
sm = plt.cm.ScalarMappable(cmap=cmap_dist, norm=divnorm)
cbar = fig.colorbar(sm, cax=cbar_ax)
cbar.set_label('Distances', fontsize=fs)
cbar.ax.tick_params(labelsize=24)

plt.tight_layout()

plt.savefig('fig1.pdf', bbox_inches='tight')
plt.show()

RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmpymfr7e_6 e2c20be248d0d30b3ff7c090bda2ce07.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.24 (TeX Live 2022/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./e2c20be248d0d30b3ff7c090bda2ce07.tex
LaTeX2e <2022-11-01> patch level 1
L3 programming layer <2023-01-16>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2022/07/02 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)

! LaTeX Error: File `type1ec.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.9 \usepackage
               [utf8]{inputenc}^^M
No pages of 

RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmp84ucyvi8 e2c20be248d0d30b3ff7c090bda2ce07.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.24 (TeX Live 2022/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./e2c20be248d0d30b3ff7c090bda2ce07.tex
LaTeX2e <2022-11-01> patch level 1
L3 programming layer <2023-01-16>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2022/07/02 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)

! LaTeX Error: File `type1ec.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.9 \usepackage
               [utf8]{inputenc}^^M
No pages of 

<Figure size 3500x1000 with 14 Axes>